## Block matrix multiplication using threading

In [1]:
import numpy as np
import time


#### This code is for helping you 

In [2]:
"""
   Create a matrix with size mxn- Return random integers from `low`=-100 `high`=100.
   :param m: number of rows
   :param n:number of columns
   :return: generated random matrix 
"""
np.random.seed=32
def createMatrix(m,n):
    ## write your code here
    return np.random.randint(100,size=(m,n))
  
"""
Create a matrix with size mxn- Return random integers from `low`=-100 `high`=100.
   :param A: Matrix A
   :param B:number of columns
   :return: C: Matrix multiplication of A and B
"""
def multiplyMatrices(A,B):
    r, c=A.shape
    r1,c1=B.shape
    C=np.empty((r,c1))
    np.matmul
    for i in range(r): 
        for j in range(c): 
            for k in range(c): 
                C[i][j] += A[i][k] * B[k][j]
    return C

"""
This method aims to check if the multiplication result using threading and blocks is equal to multiplication 
result using np.matmul
   :param C1: is matrix which is the results of AxB using block multiplciaton
   :param C2:is matrix which is the results of AxB using np.matmul to validate our solution
"""
def check_the_solution(C1,C2):
    
    anw=np.abs(np.sum(C2-C1))
    print(anw)
    if anw==0:
        print('answer is correct')
    else:
        print('answer is wrong')

In [3]:
AA=createMatrix(4,4)
BB=createMatrix(4,4)
CC1=multiplyMatrices(AA,BB)
CC2=np.matmul(AA,BB)
DD=check_the_solution(CC1,CC2)

0.0
answer is correct


In [4]:
n_rows=300
AA=createMatrix(n_rows,n_rows)
BB=createMatrix(n_rows,n_rows)
t1=time.time()
CC=multiplyMatrices(AA,BB)
t2=time.time()
print(t2-t1)

195.19839763641357


In [5]:
import threading 

import time

### 1- create a matrix class that contains all information about the matrix + the function of releasing the threads

In [6]:
"""
Matrix class aims to manage all whole block multiplication 
"""
class Matrix:

    """ 
    Constructor
    A: Matrix A -- Squared matrix
    B:Matrix B -- Squared matrix
    param no_of_blocks -- Number of blocks  assume that the number of blocks are same in rows and columnsas well as A and B are squared matirces
    """

    def __init__(self, A ,B ,no_of_blocks):

        self.A= A ### write your code
        self.B= B ### write your code
        self.block_size= int(A.shape[1]/no_of_blocks) ### write your code
        self.no_of_blocks= no_of_blocks ### write your code
        
        self.blocks=np.empty((no_of_blocks, no_of_blocks), dtype=object)
        self.C= np.zeros_like(self.A) ### create matrix of zeros using numpy- write your code 
    """
    - Method aims to release the threads, which each thread caclulates one block. 
    - The result of each block must be cloned to C 

    """
    def release_threads(self):
        ## release threads === nested loops
        for i in range(self.no_of_blocks):
          for j in range (self.no_of_blocks):
            B1 = Block(i, j, self.block_size, self.A, self.B, self.no_of_blocks)
            B1.thread.start()
            self.blocks[i][j]=B1 


        ## join the threads + return the result of block_C to C   
        for i in range(self.no_of_blocks):
          for j in range (self.no_of_blocks):
            self.C[i*self.block_size: i*self.block_size+self.block_size, j*self.block_size: j*self.block_size+self.block_size]=self.blocks[i][j].block_C



### 1- Create a class with name Block. This class contains all attributes and methods of block. For example, location of the blocks etc. 

In [7]:
class Block(object):
    """
    Constructor 
    id_x: id of block in rows (x axis)
    id_y: id of block in colunm (y axis)
    block_size: the block size
    A: A matrix
    B: B matrix
    no_of_blocks: number of blocks is needed to copy the blocks
    
    """
    def __init__(self, id_x,id_y,block_size, A, B, no_of_blocks):
        self.id_x= id_x ### write your code
        self.id_y= id_y ### write your code
        self.no_of_blocks= no_of_blocks ### write your code
        
        self.start_row= id_x*block_size ### write your code
        self.end_row= self.start_row+block_size  ### write your code
        
        self.start_column= id_y*block_size### write your code
        self.end_column= self.start_column+block_size ### write your code
        ##print(self.start_row,self.end_row,self.start_column,self.end_column)
        
        self.block_size= block_size ### write your code
        self.block_C=  np.zeros_like((block_size, block_size)) ### 
        ## thread varaible to be used in starting the cacluculation of the blocks
        self.thread=threading.Thread(target=self.Calculate_block_multiplication,args=(A,B))
        
    """
    Method to multiply two matrices 
    A_block: first matrix
    B_block: second matrix
    Return the multiplication result of two blocks 
    """
    def multiplyMatrices(self,A_block,B_block):
     
        r, c=A_block.shape
        r1,c1=B_block.shape
        
        assert c==r1, 'number of columns in matrix A must equal to number of rows in matrix B'
        block_C=np.empty((r,c1))

        for i in range(r): 
            for j in range(c): 
                for k in range(c): 
                    block_C[i][j] += A_block[i][k] * B_block[k][j]

        return np.matmul(A_block,B_block)
    
    """ 
    Method aims to copy the block from the original matrix , it requires the id_r and id_c to know which block
    param AA: matrix 
    param i_r, the id of block in rows
    param j_c, the id of block in columns
    Return the copyed block
    """
    def copy_block_from_matrix(self, AA,i_r, j_c):
            ### write your code
            self.start_row= i_r*self.block_size
            self.end_row= self.start_row+self.block_size
            self.start_column= j_c*self.block_size### write your code
            self.end_column= self.start_column+self.block_size
            return AA[self.start_row:self.end_row, self.start_column:self.end_column]
    """
    Methods aims to copy each two corresponding blocks in A and B through loop- For example
    C_Block(0,0)= A_Block(0,0)× B_Block(0,0) + A_Block(0,1)× B_Block(1,0) + A_Block(0,2)× B_Block(2,0) 
    Therefore, this method makes the following:
    1- call copy_block_from_matrix to copy two blocks
    2- call multiplyMatrices to multiply two blocks
    3- add the result to block_C
    """
    def Calculate_block_multiplication(self,A,B):
        print('Thread', self.id_x,self.id_y, ' released')
        ## write your code
        for i in range(self.no_of_blocks):
          A1 = self.copy_block_from_matrix(A, self.id_x, i)
          B1 = self.copy_block_from_matrix(B, i, self.id_y)
          C1 = self.multiplyMatrices(A1, B1)
          self.block_C = np.add(self.block_C, C1)   


### 2- Apply threading to perform block matrix multiplication, where the matrix size =600x600

### 3- Compare the execution-time of different size of blocks 

In [8]:

A=createMatrix(4,4)
B=createMatrix(4,4)
print(A)
print()
print(B)


[[34 71 10 34]
 [20 71 52 74]
 [59 78 87  3]
 [29 91 80 82]]

[[ 2 70 65 65]
 [33 85 48 88]
 [27 72 61 52]
 [25 48 74 80]]


In [9]:
mm=Matrix(A,B,2)
mm.release_threads()
print()
print()
print(mm.blocks[0,0].block_C)


Thread 0 0  released
Thread 0 1  released
Thread 1 0  released
Thread 1 1  released


[[ 3531 10767]
 [ 5637 14731]]


In [10]:
C10=np.matmul(mm.A,mm.B)
print(C10)
print()
print(mm.C)

[[ 3531 10767  8744 11698]
 [ 5637 14731 13356 16172]
 [ 5116 17168 13108 15463]
 [ 7271 19461 17201 20613]]

[[ 3531 10767  8744 11698]
 [ 5637 14731 13356 16172]
 [ 5116 17168 13108 15463]
 [ 7271 19461 17201 20613]]


In [11]:
check_the_solution(C10,mm.C)

0
answer is correct
